In [3]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.font_manager import FontProperties
from collections import Counter
from itertools import combinations
import numpy as np

# --- 参数与资源设置 ---
font_path = '/kaggle/input/nssfc-2025/simkai.ttf'
csv_path = '/kaggle/input/nssfc-2025/nssfc-fenci.csv'
TOP_K_EDGES = 150

# 1. 研究元词汇与方法论相关
research_meta_stopwords = [
    '影响', '体系', '视域', '机理', '效应', '构建', '建构', '理论', '实践', 
    '对策', '视角', '背景', '基于', '模式', '应用', '问题', '评估', '测度', 
    '关系', '形成', '重构', '逻辑', '形态', '系统', '模型', '演化', '演变', 
    '经验', '资料', '史料', '要素', '研究', '机制', '路径', '及其'
]

# 2. 高度泛化的背景词
generalized_context_stopwords = [
    '中国', '社会', '我国', '国家', '时代', '当代', '时期', '历史', 
    '世纪', '1949', '近代'
]

# 3. 抽象的过程、目标与修饰词
abstract_process_stopwords = [
    '策略', '提升', '优化', '整理', '驱动', '赋能', '建设', '实现', '应对', 
    '促进', '困境', '动态', '识别', '效能', '交融'
]

custom_stopwords = set(research_meta_stopwords+generalized_context_stopwords+abstract_process_stopwords)


# --- 数据加载与网络构建 ---
df = pd.read_csv(csv_path)
df = df[df['涉及学科'] == '管理学'].copy()

# 修正列名为 '项目名称分词列'
df.dropna(subset=['课题名称分词'], inplace=True)
project_keywords_list = [text.split() for text in df['课题名称分词']]

co_occurrences = []
for keywords in project_keywords_list:
    keywords = [word for word in keywords if word not in custom_stopwords and len(word) > 1]
    for pair in combinations(sorted(keywords), 2):
        co_occurrences.append(pair)
co_occurrence_counts = Counter(co_occurrences)


# ===== 新增：打印 & 导出 TOP_K_EDGES =====
top_k_edges = co_occurrence_counts.most_common(TOP_K_EDGES)
if top_k_edges:
    print(f"\n前 {len(top_k_edges)} 条高频共现边（格式：（词语1，词语2，共现频率））：")
    for (w1, w2), c in top_k_edges:
        print(f"（{w1}，{w2}，{c}）")


G = nx.Graph()
edges_to_add = [(word1, word2, {'weight': count}) 
                for (word1, word2), count in co_occurrence_counts.most_common(TOP_K_EDGES)]
G.add_edges_from(edges_to_add)

# --- 可视化网络图 ---
plt.figure(figsize=(24, 24)) 
pos = nx.spring_layout(G, k=0.9, iterations=50, seed=42)

node_degrees = dict(G.degree())
node_sizes = [v * 200 for v in node_degrees.values()]
NODE_COLOR = '#fdc086'

# --- 按频率区间为边分类并着色 ---
edge_weights = [data['weight'] for u, v, data in G.edges(data=True)]

try:
    categories, bins = pd.qcut(edge_weights, q=3, labels=False, retbins=True, duplicates='drop')
except ValueError:
    categories = [0] * len(edge_weights)
    bins = [min(edge_weights), max(edge_weights)]
    
color_palette = ['#4daf4a', '#377eb8', '#e41a1c'] 
edge_colors = [color_palette[cat] for cat in categories]

# 绘制节点和边
nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=NODE_COLOR, alpha=0.95)
nx.draw_networkx_edges(G, pos, width=2.0, edge_color=edge_colors) 

# --- 核心修改：将字号整合进FontProperties ---
# 为图例标题和条目分别创建包含了字号信息的字体对象
legend_title_font = FontProperties(fname=font_path, size=30)
legend_prop_font = FontProperties(fname=font_path, size=26)

legend_patches = []
for i in range(len(bins) - 1):
    label = f'频率: {int(bins[i])}-{int(bins[i+1])}'
    patch = mpatches.Patch(color=color_palette[i], label=label)
    legend_patches.append(patch)

# 移除冲突的fontsize和title_fontsize，使用新的字体对象
plt.legend(handles=legend_patches, 
           prop=legend_prop_font,          # <--- 使用包含字号的字体对象
           title="共现频率区间", 
           title_fontproperties=legend_title_font, # <--- 使用包含字号的字体对象
           loc='upper right',
           bbox_to_anchor=(1.1, 1.0))
# ------------------------------------

# --- 节点文字绘制 (无变动) ---
# 为节点文字创建总的字体对象
font_prop = FontProperties(fname=font_path)
MIN_FONT_SIZE, MAX_FONT_SIZE = 26, 40
min_degree, max_degree = min(node_degrees.values()), max(node_degrees.values())
if max_degree == min_degree:
    normalized_degrees = {node: 0.5 for node in node_degrees}
else:
    normalized_degrees = {
        node: (degree - min_degree) / (max_degree - min_degree)
        for node, degree in node_degrees.items()
    }

for node, (x, y) in pos.items():
    font_size = MIN_FONT_SIZE + normalized_degrees[node] * (MAX_FONT_SIZE - MIN_FONT_SIZE)
    plt.text(x, y, node, 
             fontproperties=font_prop, 
             fontsize=font_size, 
             ha='center', 
             va='center',
             fontweight='bold')

# --- 图表标题和保存 (无变动) ---
plt.title('管理学项目关键词共现网络图 (Top 150)', fontproperties=font_prop, fontsize=35)
plt.axis('off')
plt.tight_layout()

plt.savefig('keyword_co-occurrence_network_management.png', dpi=300, bbox_inches='tight')
print("\n关键词共现网络图已保存为 'keyword_co-occurrence_network_management.png'")


前 150 条高频共现边（格式：（词语1，词语2，共现频率））：
（人工，智能，76）
（协同，治理，33）
（企业，创新，31）
（治理，风险，24）
（企业，技术，19）
（企业，治理，19）
（产业，创新，18）
（企业，协同，18）
（人工，生成式，18）
（智能，生成式，18）
（智能，治理，16）
（企业，政策，15）
（创新，技术，14）
（人工，企业，14）
（企业，智能，14）
（人工，治理，14）
（企业，链主，14）
（协同，政策，14）
（数字，治理，14）
（关键，技术，13）
（技术，核心，13）
（技术，突破，13）
（企业，制造，13）
（企业，转型，13）
（企业，韧性，13）
（企业，突破，12）
（创新，协同，12）
（低空，经济，12）
（创新，生态，12）
（企业，关键，11）
（关键，核心，11）
（企业，数据，11）
（创新，治理，11）
（价值，生态，11）
（企业，数字，11）
（智能，风险，11）
（政策，智能，11）
（政策，数字，11）
（企业，核心，10）
（耐心，资本，10）
（创新，智能，10）
（创新，政策，10）
（人工，风险，10）
（人工，政策，10）
（发展，质量，10）
（企业，绿色，10）
（产业，生态，10）
（城市，治理，10）
（平台，治理，10）
（人工，创新，9）
（专精，企业，9）
（政策，生态，9）
（数字，生态，9）
（数字化，转型，9）
（协同，发展，9）
（绿色，转型，9）
（协同，风险，9）
（数据，治理，9）
（治理，韧性，8）
（技术，数智，8）
（关键，突破，8）
（核心，突破，8）
（企业，风险，8）
（企业，资本，8）
（企业，发展，8）
（新质，生产力，8）
（创新，突破，8）
（供应，链韧性，8）
（技术，智能，8）
（治理，生态，8）
（治理，生成，8）
（中小，企业，8）
（企业，数字化，8）
（协同，智能，8）
（文化，遗产，8）
（企业，行为，8）
（治理，行为，8）
（供应链，协同，8）
（技术，政策，7）
（技术，治理，7）
（产业，政策，7）
（产业，技术，7）
（创新，融合，7）
（技术，融合，7）
（产业，企业，7）
（创新，数字，7）
（政策，数据，7）
（人工，大模型，7）
（大模型，智能，7）
（安全，风险，7）
（人工，技术，7

In [4]:
!pip install -q pyvis

import pandas as pd
import networkx as nx
from pyvis.network import Network
from collections import Counter
from itertools import combinations
from IPython.display import IFrame, display

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 13.7 MB/s eta 0:00:0000:01


In [5]:
TOP_K_EDGES = 300

top_k_edges = co_occurrence_counts.most_common(TOP_K_EDGES)

if not top_k_edges:
    raise RuntimeError("未产生任何共现边（可能是分词列为空或停用词过严）。")

# --- 使用 NetworkX 构建图 ---
G = nx.Graph()
G.add_edges_from([(w1, w2, {"weight": c}) for (w1, w2), c in top_k_edges])

# 节点/边属性
node_degrees = dict(G.degree())
edge_weights = [G[u][v]["weight"] for u, v in G.edges()]

# 安全范围计算
MIN_NODE_SIZE, MAX_NODE_SIZE = 30, 50
MIN_LABEL_SIZE, MAX_LABEL_SIZE = 12, 28
MIN_EDGE_WIDTH, MAX_EDGE_WIDTH = 1, 10

min_degree = min(node_degrees.values()) if node_degrees else 0
max_degree = max(node_degrees.values()) if node_degrees else 1
min_weight = min(edge_weights) if edge_weights else 1
max_weight = max(edge_weights) if edge_weights else 1

# === 新增：复用“静态图”配色方案 ===
import pandas as pd
palette = ['#4daf4a', '#377eb8', '#e41a1c']  # 绿/蓝/红（与静态一致）
weights_in_order = [c for (_, _), c in top_k_edges]

# 三分位分箱，得到每条边所在分箱（与 top_k_edges 顺序对应）
try:
    cats, bins = pd.qcut(weights_in_order, q=3, labels=False, retbins=True, duplicates='drop')
except ValueError:
    # 权重全相同时 qcut 可能失败：全部归为一类
    cats = [0] * len(weights_in_order)
    bins = [min(weights_in_order), max(weights_in_order)]

# 构建颜色字典：以排序后的节点对作为 key，保证查找稳定
edge_color_dict = {}
for ((u, v), w), c in zip(top_k_edges, cats):
    key = tuple(sorted((u, v)))
    edge_color_dict[key] = palette[int(c)]

# --- PyVis 交互式可视化 ---
net = Network(
    height="820px",
    width="100%",
    notebook=True,
    heading="关键词共现网络图（交互版）",
    directed=False
)

# 节点（与静态图一致的节点色）
NODE_COLOR = "#fdc086"
for node, degree in node_degrees.items():
    # 节点圆圈大小
    if max_degree > min_degree:
        size = MIN_NODE_SIZE + (degree - min_degree) * (MAX_NODE_SIZE - MIN_NODE_SIZE) / (max_degree - min_degree)
    else:
        size = (MIN_NODE_SIZE + MAX_NODE_SIZE) / 2

    # 同步映射标签字号
    if MAX_NODE_SIZE > MIN_NODE_SIZE:
        label_size = MIN_LABEL_SIZE + (size - MIN_NODE_SIZE) * (MAX_LABEL_SIZE - MIN_LABEL_SIZE) / (MAX_NODE_SIZE - MIN_NODE_SIZE)
    else:
        label_size = (MIN_LABEL_SIZE + MAX_LABEL_SIZE) / 2

    net.add_node(
        node,
        label=node,
        size=float(size),
        color=NODE_COLOR,  # ★ 与静态图一致
        title=f"关键词：{node}\n连接数：{degree}",
        font={"size": int(label_size), "bold": True, "strokeWidth": 2, "strokeColor": "#ffffff"}
    )

# 边（三分位颜色 + 宽度映射）
for u, v, data in G.edges(data=True):
    w = data["weight"]
    if max_weight > min_weight:
        width = MIN_EDGE_WIDTH + (w - min_weight) * (MAX_EDGE_WIDTH - MIN_EDGE_WIDTH) / (max_weight - min_weight)
    else:
        width = (MIN_EDGE_WIDTH + MAX_EDGE_WIDTH) / 2

    key = tuple(sorted((u, v)))
    ecolor = edge_color_dict.get(key, palette[0])

    net.add_edge(
        u, v,
        width=float(width),
        value=int(w),               # 用真实权重参与物理布局
        color=ecolor,               # ★ 与静态图一致的边颜色
        title=f"{{{u}，{v}}} 共现频率：{w}"
    )

# 交互/物理参数（JSON 字符串）
net.set_options(
    """
    {
      "layout": { "improvedLayout": true },
      "interaction": { "hover": true, "tooltipDelay": 120, "zoomView": true, "dragNodes": true },
      "configure": { "enabled": true, "filter": ["physics"] },
      "nodes": {
        "font": { "face": "SimKai, KaiTi, Microsoft YaHei, Arial", "size": 18 }
      },
      "edges": {
        "smooth": { "type": "dynamic" }
      },
      "physics": {
        "enabled": true,
        "solver": "barnesHut",
        "stabilization": { "enabled": true, "iterations": 1600, "updateInterval": 25, "fit": true },
        "timestep": 0.35,
        "maxVelocity": 20,
        "minVelocity": 0.75,
        "barnesHut": {
          "gravitationalConstant": -3800,
          "centralGravity": 0.28,
          "springLength": 135,
          "springConstant": 0.018,
          "damping": 0.72,
          "avoidOverlap": 0.25
        }
      }
    }
    """
)

# （可选）右上角“伪图例”
legend_y0 = -420  # 顶部 y 基准（随画布大小可微调）
for i in range(len(bins) - 1):
    label = f"频率: {int(bins[i])}-{int(bins[i+1])}"
    net.add_node(
        f"_legend_{i}", label=label, shape="box", color=palette[i % len(palette)],
        x=900, y=legend_y0 + i * 60, fixed={"x": True, "y": True}, physics=False,
        font={"size": 18, "bold": True, "color": "#222"}
    )

# 保存并在 Notebook 中显示
html_path = "interactive_keyword_network_management.html"
net.show(html_path)  # 会保存并在 Notebook 中内嵌（notebook=True）

print(f"动态交互式网络图（与静态配色一致）已生成：{html_path}")
display(IFrame(src=html_path, width="100%", height=840))

interactive_keyword_network_management.html
动态交互式网络图（与静态配色一致）已生成：interactive_keyword_network_management.html
